In [21]:
class CanNotOrderAdjacentElements(Exception):
    pass

def is_consec(ordering, subset):
    if len(subset)==0:
        return True
    
    indexes = list(map(ordering.index, subset))
    diff = max(indexes)-min(indexes)
    return diff==len(subset)-1
    
def check_order(connecteds, ordering):
    return all([is_consec(ordering,cc) for cc in connecteds])

def get_order(connecteds):
    from itertools import permutations
    from functools import reduce
    alphabet = reduce(set.union, connecteds, set())
    for order in permutations(alphabet):
        if check_order(connecteds,order):
            return order
    #Otherwise:
    raise(CanNotOrderAdjacentElementsError())
    

In [259]:
from functools import reduce
def mapreduce(reduction, mapping, initaliser, collection):
    mapped = map(mapping, collection)
    print(type(mapped))
    print(type(list(mapped)[0]))
    
    return reduce(reduction,mapped,initaliser)

In [86]:
import pylatex
from pylatex import Document, Section, Subsection, Subsubsection, Table
from lists import Description

from datetime import timedelta, time


In [177]:
#CONFIG
import pytz
LOCAL_TIMEZONE = pytz.timezone("Australia/Perth")

SLOT_LENGTH_MINUTES = 30


In [224]:
def get_slots(start,end, exclusive_end=True, slot_length_minutes = SLOT_LENGTH_MINUTES):
    from dateutil import rrule
    until = end-timedelta(0,0,1) if exclusive_end else end
    slots = rrule.rrule(rrule.MINUTELY, 
                          interval = slot_length_minutes,
                          dtstart = start,
                          until = until,
                          )
    return list(slots)
    

In [312]:
def parse_datetime(datetime_str):
    import dateutil.parser
    
    raw = dateutil.parser.parse(datetime_str)
    return raw.astimezone(LOCAL_TIMEZONE)

def normalise(rawstr): 
    import unicodedata
    import pylatex.utils
    
    str_escaped = str(pylatex.utils.escape_latex(rawstr))
    return str_escaped.strip()
    

def load_list(csl):
    ldlst =  str.split(normalise(csl), ', ')
    if ldlst==['']:
        return []
    else:
        return ldlst

class session (object):
    def __init__(self, id,
                 start_time_str,end_time_str,
                 title,tags_str,people_str,
                 venues_str,description):
        self.id = int(id)
        self.start = parse_datetime(start_time_str)
        self.end = parse_datetime(end_time_str)
        self.title = normalise(title)
        self.tags = load_list(tags_str)
        self.people = load_list(people_str)
        self.venues = load_list(venues_str)
        self.description = normalise(description)
    
    @property
    def day(self):
        week_days   = ['Sunday', 
          'Monday', 
          'Tuesday', 
          'Wednesday', 
          'Thursday',  
          'Friday', 
          'Saturday']
        daynum = self.start.weekday()
        return week_days[daynum]
    
    
    @property
    def start_time(self):
        return self.start.strftime("%H:%M")
    
    @property
    def end_time(self):
        return self.end.strftime("%H:%M")
    
    @property
    def timeslots(self, slot_length_minutes=SLOT_LENGTH_MINUTES):
        return get_slots(self.start,self.end, slot_length_minutes)
        
            

In [313]:
import csv

sessions = []
with open('con_data/swancon_2014.csv', 'r') as csvfile:
    con_csv = csv.reader(csvfile)
    next(con_csv) #Skip heading
    sessions = [session(*row) for row in con_csv]

sessions.sort(key=lambda ss: ss.start)



In [ ]:
from collections import OrderedDict, Callable

class DefaultOrderedDict(OrderedDict):
    # Source: http://stackoverflow.com/a/6190500/562769
    def __init__(self, default_factory=None, *a, **kw):
        if (default_factory is not None and
           not isinstance(default_factory, Callable)):
            raise TypeError('first argument must be callable')
        OrderedDict.__init__(self, *a, **kw)
        self.default_factory = default_factory

    def __getitem__(self, key):
        try:
            return OrderedDict.__getitem__(self, key)
        except KeyError:
            return self.__missing__(key)

    def __missing__(self, key):
        if self.default_factory is None:
            raise KeyError(key)
        self[key] = value = self.default_factory()
        return value

    def __reduce__(self):
        if self.default_factory is None:
            args = tuple()
        else:
            args = self.default_factory,
        return type(self), args, None, None, self.items()

    def copy(self):
        return self.__copy__()

    def __copy__(self):
        return type(self)(self.default_factory, self)

    def __deepcopy__(self, memo):
        import copy
        return type(self)(self.default_factory,
                          copy.deepcopy(self.items()))

    def __repr__(self):
        return 'OrderedDefaultDict(%s, %s)' % (self.default_factory,
                                               OrderedDict.__repr__(self))

In [314]:
from collections import defaultdict
dtv_bookings = defaultdict(lambda : defaultdict(lambda : defaultdict(str)))

for session in sessions:
    for slot in session.timeslots:
        for venue in session.venues:
            dtv_bookings[session.day][slot][venue] = session.title


In [315]:
class SessionOutOfSlotAlignmentError(Exception):
    def __init__(self, slot, day):
        self.slot=slot
        self.day=day
        

In [ ]:
ordered_days

In [317]:


venue_columns = list(get_order([session.venues for session in sessions]))
slots = None
doc = Document()
with doc.create(Section('Timetable')):
    for date in ordered_days:
        if len(dtv_bookings[day])>0:
            with doc.create(Subsection(day)):
                table_spec = "c|"+"c|"*len(venue_columns)
                with doc.create(Table(table_spec)) as timetable:
                    timetable.add_row(["Time"]+venue_columns)
                    timetable.add_hline()
                    timetable.add_hline()

                    used_slots = set(dict.keys(dtv_bookings[day]))
                    slots = get_slots(min(used_slots),max(used_slots), exclusive_end=False)

                    for used_slot in used_slots:
                        if not used_slot in slots:
                            raise SessionOutOfSlotAlignmentError(used_slot,day)

                    for slot in slots:
                        venue_bookings = dtv_bookings[day][slot]
                        venue_slots = [venue_bookings[vv] for vv in venue_columns]

                        slot_time_str = slot.strftime("%H:%M")
                        timetable.add_row([slot_time_str] + venue_slots)


with doc.create(Section('Program')):
    for day_name, day_session in groupby(sessions, lambda ss: ss.day):
        with doc.create(Subsection(day_name)):
            for session in day_session:
                with doc.create(Subsubsection(session.title)):
                    with doc.create(Description()) as desc:
                        desc.additem("Time:", session.start_time+" -- "+session.end_time)
                        
                        if len(session.venues)>0:
                            desc.additem("Venue:", ", ".join(session.venues))
                        if len(session.people)>0:
                            desc.additem("People:", ", ".join(session.people))
                        if len(session.tags)>0:
                            desc.additem("Tags:", ", ".join(session.tags))
                    doc.append(session.description)
                        


NameError: name 'ordered_days' is not defined

In [294]:
from IPython.display import FileLink, FileLinks
with open("out/temp.tex", 'w') as temp_out:
    doc.dump(temp_out)

!lualatex --output-directory=out temp.tex
FileLinks("./out")

This is LuaTeX, Version beta-0.70.2-2012070300 (TeX Live 2012/Debian)
 restricted \write18 enabled.
(./out/temp.tex
LaTeX2e <2011/06/27>
LuaTeX adaptation of babel <v3.8m-luatex-1.5> and hyphenation patterns for engl
ish, dumylang, nohyphenation, ethiopic, farsi, arabic, pinyin, croatian, bulgar
ian, ukrainian, russian, slovak, czech, danish, dutch, usenglishmax, ukenglish,
 finnish, french, basque, ngerman, german, swissgerman, ngerman-x-2012-05-30, g
erman-x-2012-05-30, monogreek, greek, ibycus, ancientgreek, hungarian, bengali,
 tamil, hindi, telugu, gujarati, sanskrit, malayalam, kannada, assamese, marath
i, oriya, panjabi, italian, latin, latvian, lithuanian, mongolian, mongolianlmc
, nynorsk, bokmal, indonesian, esperanto, coptic, welsh, irish, interlingua, se
rbian, serbianc, slovenian, friulan, romansh, estonian, romanian, armenian, upp
ersorbian, turkish, afrikaans, icelandic, kurmanji, polish, portuguese, galicia
n, catalan, spanish, swedish, thai, loaded.
(/usr/share/texlive

./out/
  temp.log
  temp.tex
  temp.aux
  temp.pdf